In [1]:
# Import all the necessary libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from typing import Callable
import cv2 as cv
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects
import pandas as pd

In [2]:
# Import our own modules
from utils.coin import *

%load_ext autoreload
%autoreload 2

In [3]:
hand_images_list = os.listdir('../data/train/3. hand')
hand_images_path = '../data/train/3. hand'
hand_outliers_images_list = os.listdir('../data/train/6. hand_outliers')
hand_outliers_images_path = '../data/train/6. hand_outliers'
neutral_images_list = os.listdir('../data/train/1. neutral_bg')
neutral_images_path = '../data/train/1. neutral_bg'
neutral_images_outliers_list = os.listdir('../data/train/4. neutral_bg_outliers')
neutral_images_outliers_path = '../data/train/4. neutral_bg_outliers'
noisy_images_list = os.listdir('../data/train/2. noisy_bg')
noisy_images_path = '../data/train/2. noisy_bg'
noisy_images_outliers_list = os.listdir('../data/train/5. noisy_bg_outliers')
noisy_images_outliers_path = '../data/train/5. noisy_bg_outliers'
test_coin_list = os.listdir('../data/train/test_coin')
test_coin_path = '../data/train/test_coin'

hand_images = load_images(hand_images_list, hand_images_path)
hand_outliers_images = load_images(hand_outliers_images_list, hand_outliers_images_path)
neutral_images = load_images(neutral_images_list, neutral_images_path)
neutral_images_outliers = load_images(neutral_images_outliers_list, neutral_images_outliers_path)
noisy_images = load_images(noisy_images_list, noisy_images_path)
noisy_images_outliers = load_images(noisy_images_outliers_list, noisy_images_outliers_path)

# test_coin_images = load_images(test_coin_list, test_coin_path)

# test_images_list = os.listdir('../data/test2/')
# test_images_list = [image for image in test_images_list if '.JPG' in image]
# test_images_path = '../data/test2/'
# test_images = load_images(test_images_list, test_images_path)

In [4]:
def enhance_blue_channel(image):
    """
    Enhance the blue channel and suppress other channels.

    Args
    ----
    image: np.ndarray (M, N, 3)
        BGR image of size MxN.

    Return
    ------
    enhanced_image: np.ndarray (M, N, 3)
        Image with enhanced blue channel
    """
    enhanced_image = image.copy()
    enhanced_image[:,:,0] = np.clip(image[:,:,0] * 1.3, 0, 255)  # Enhance Blue channel
    enhanced_image[:,:,2] = np.clip(image[:,:,2] * 1, 0, 255)  # Suppress Red channel
    enhanced_image[:,:,1] = np.clip(image[:,:,1] * 1, 0, 255)  # Suppress Green channel

    # detect the contours with canny
    # gray = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    # edges = cv2.Canny(gray, 200, 300)
    # contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # cv2.drawContours(enhanced_image, contours, -1, (0, 0, 0), 2)

    return enhanced_image

def calculate_compactness(contour):
    perimeter = cv.arcLength(contour, True)
    area = cv.contourArea(contour)
    if area == 0:
        return 0
    compactness = (perimeter ** 2) / (4 * np.pi * area)
    return compactness

def generate_mask(img):

    image_mask = img.copy()
    image_mask = cv.cvtColor(image_mask, cv.COLOR_RGB2BGR)

    lower_blue_range = [30, 80, 80]  # Lower bound of the HSV range for blue 20, 80, 80
    upper_blue_range = [160, 255, 255]  # Upper bound of the HSV range for blue
    
    enhanced_image = enhance_blue_channel(image_mask)
    
    # Convert to HSV color space
    #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper range of the blue color in HSV
    lower_range = np.array(lower_blue_range, dtype=np.uint8)
    upper_range = np.array(upper_blue_range, dtype=np.uint8)
    
    # Create a mask that identifies the blue coins
    mask = cv.inRange(enhanced_image, lower_range, upper_range)

    # Apply opening to the mask
    mask = np.uint8(remove_objects(mask, 100) * 255)

    kernel = np.ones((2, 2), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # apply closing to the mask
    kernel = np.ones((7, 7), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)

    # Apply Gaussian blur to the mask
    mask = cv.GaussianBlur(mask, (13, 13), 0)

    circles = cv.HoughCircles(mask, cv.HOUGH_GRADIENT, dp=1, minDist=50, param1=10, param2=25, minRadius=40, maxRadius=120) # 32
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv.circle(image_mask, (i[0], i[1]), i[2], (0, 0, 0), 2)
    
    image_mask = cv.cvtColor(image_mask, cv.COLOR_BGR2RGB)

    return image_mask


def sas(idx, img, size = 45, sigma = 2.7, thres1 = 247, thres2 = 6):
    title = ""

    blue_std = np.std(img[:,:,0])

    std = np.std(cv.cvtColor(255 - img, cv.COLOR_BGR2GRAY))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (0,0), fx=0.25, fy=0.25)
    img_copy = img.copy()

    # canny edge detection
    img_edges_std = np.std(cv.Canny(cv.cvtColor(img, cv.COLOR_BGR2GRAY), 50, 200))
    img_edges_mean = np.mean(cv.Canny(cv.cvtColor(img, cv.COLOR_BGR2GRAY), 50, 200))

    # calculate the compactness of the image
    img_edges = cv.Canny(cv.cvtColor(img, cv.COLOR_BGR2GRAY), 50, 200)
    contours, _ = cv.findContours(img_edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    compactness = 0
    for contour in contours:
        compactness += calculate_compactness(contour)
    

    red_mean = np.mean(img_copy[:,:,0]) 
    blue_mean = np.mean(img_copy[:,:,2])
    green_mean = np.mean(img_copy[:,:,1])

    if std < 18: #neutral
        title = "Neutral"
        remove_objects_size = 10
        thres2 = 1
        sigma = 2.7
        p2 = 32
        th_sigma = 2.5
        open_th = 3
    
    else: #hand
        remove_objects_size = 120
        if img_edges_std < 50:
            title = "Hand"
            thres2 = 3
            sigma = 2.7
            p2 = 38
            th_sigma = 2.7 # 2.7
            open_th = 6 # 6
        else: # noisy
            title = "Noisy"
            thres2 = 2 # 1
            sigma = 2.7 # 2.7
            p2 = 32 # 30
            th_sigma = 2.2 # 2.2
            open_th = 5 # 3
    
    if title == "Noisy":
        img = generate_mask(img)

    #print(idx, " -", title, ": edges_std:", img_edges_std, "edges_mean:", img_edges_mean, "compactness:", compactness)

    img[:,:,0] = img[:,:,0]*0.25 #red
    img[:,:,1] = img[:,:,1]*0.25 #green
    img[:,:,2] = img[:,:,2]*1 #blue 

    imgGray = cv.cvtColor(255 - img, cv.COLOR_BGR2GRAY)

    low_pass = cv.GaussianBlur(imgGray, (size, size), sigma)
    high_pass = cv.subtract(imgGray, low_pass)
    inverted_HP = (255 - high_pass)
    _, inverted_thresholded = cv.threshold(inverted_HP, thres1, 255, cv.THRESH_BINARY)
    _, thresholded = cv.threshold(high_pass, thres2, 255, cv.THRESH_BINARY)

    #thresholded = np.uint8(remove_objects(thresholded, 16) * 255)
    thresholded = np.uint8(remove_objects(thresholded,remove_objects_size) * 255)
    thresholded_open = apply_closing(thresholded, open_th) # 3
    thresholded_open = cv.GaussianBlur(thresholded_open, (size, size), th_sigma) # 2

    circles = cv.HoughCircles(thresholded_open, cv.HOUGH_GRADIENT, dp=1, minDist=50, param1=15, param2=p2, minRadius=40, maxRadius=120) # 32
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv.circle(img_copy, (i[0], i[1]), i[2], (0, 0, 0), 10)

    display_img_side(thresholded_open, img_copy, idx, title)

    return [img_edges_mean, img_edges_std, compactness]

# set the default values for the parameters in the interactive plot
#interact(sas, size=(25, 51, 2), sigma=(0, 2, 0.1), thres1=(200, 255, 1), thres2=(1, 20, 1)) 

# apply the function to all the images in the folder
def process_images(images, range):
    means = []
    stds = []
    mean = []
    std = []
    compact = []
    count = 0
    for idx, image in enumerate(images):
        if count < range[0]:
            count += 1
            continue
        img_RGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        info = sas(idx,image)
        if count == range[1]:
            break
        count += 1
        mean.append(info[0])
        std.append(info[1])
        compact.append(info[2])
        imgGray = cv.cvtColor(255 - image, cv.COLOR_BGR2GRAY) 
        means.append(np.mean(imgGray))
        stds.append(np.std(imgGray))
    df_means_stds = pd.DataFrame({'mean': means, 'std': stds, 'mean': mean, 'std': std, 'comp': compact})
    return df_means_stds

# 45 1.7 247 12

In [5]:
df_natural = process_images(neutral_images, [0, 50])
df_hand = process_images(hand_images, [0, 50])
df_noisy = process_images(noisy_images, [0, 50])

df_natural_outliers = process_images(neutral_images_outliers, [0, 50])
df_hand_outliers = process_images(hand_outliers_images, [0, 50])
df_noisy_outliers = process_images(noisy_images_outliers, [0, 50])

In [8]:
display(df_natural.describe())
display(df_hand.describe())
display(df_noisy.describe())
display(df_natural_outliers.describe())
display(df_hand_outliers.describe())
display(df_noisy_outliers.describe())

,mean,std,comp
count,16.000000,16.000000,16.000000
mean,3.132016,27.976793,16518.562586
std,0.567632,2.498339,2830.637422
min,2.234990,23.768199,11807.671874
25%,2.805000,26.596534,14901.277227
50%,3.032800,27.643321,16153.759350
75%,3.450150,29.459806,17770.737792
max,4.302870,32.843830,22340.837424


,mean,std,comp
count,10.000000,10.000000,10.000000
mean,2.870977,26.736643,19398.146351
std,0.654638,3.095311,5343.962311
min,1.960780,22.274520,11311.916455
25%,2.315613,24.172514,14871.766114
50%,2.934030,27.165565,20527.655156
75%,3.380365,29.164306,21594.613888
max,3.703960,30.508859,27435.450211


,mean,std,comp
count,15.000000,15.000000,15.000000
mean,25.606862,76.521178,91243.542594
std,2.719404,3.532345,6838.414239
min,21.004180,70.106279,79277.302816
25%,23.867320,74.270828,86126.492667
50%,25.694650,76.758848,92486.666008
75%,26.937435,78.376091,96277.512332
max,32.699670,85.259295,102757.730963


,mean,std,comp
count,17.000000,17.000000,17.000000
mean,1.941130,21.548088,9760.605458
std,0.910793,5.267994,5002.768570
min,0.567630,12.017631,3363.031024
25%,1.177760,17.289930,6619.559533
50%,1.891080,21.878053,8976.290232
75%,2.289900,24.055786,10584.465505
max,4.093090,32.046600,23336.247641


,mean,std,comp
count,7.000000,7.000000,7.000000
mean,3.217444,28.348051,29089.244570
std,0.606006,2.682876,7392.043646
min,2.405330,24.649007,18685.271764
25%,2.776355,26.434991,24551.782274
50%,3.256690,28.633022,27738.510881
75%,3.639105,30.242455,33851.496441
max,4.029170,31.799436,40394.371917


,mean,std,comp
count,16.000000,16.000000,16.000000
mean,37.300582,90.080437,120317.265833
std,1.752236,1.774305,4678.567840
min,33.757750,86.421297,111507.791060
25%,36.064565,88.858197,117463.374654
50%,37.733880,90.544331,119869.255339
75%,38.359693,91.160541,121319.719228
max,39.809750,92.556307,132100.203533


In [7]:
# 32  - Hand : edges_std: 48.315652089417185 edges_mean: 9.50912 compactness: 55831.59295733042
# 33  - Noisy : edges_std: 88.66054116573838 edges_mean: 35.87272 compactness: 115375.0035701889
# 34  - Neutral : edges_std: 29.044469422371936 edges_mean: 3.35223 compactness: 18262.251516879132
# 35  - Neutral : edges_std: 29.444846515198552 edges_mean: 3.44658 compactness: 31206.742880805898